In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
batch_size = 1

# Hamburger Dataset
class TrainDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./dataset.csv', delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:10])
        self.target_data = torch.from_numpy(hamdata[:, 10:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len
    
    
class TestDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./testset.csv', delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:10])
        self.target_data = torch.from_numpy(hamdata[:, 10:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len
    
class EvalDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./myinput.csv', delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:10])
        self.target_data = torch.from_numpy(hamdata[:, 10:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len
    
train_dataset = TrainDataset()
test_dataset = TestDataset()
eval_dataset = EvalDataset()

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader =  torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

eval_loader =  torch.utils.data.DataLoader(dataset=eval_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [3]:
class NNModel(torch.nn.Module):
    def __init__(self):
        super(NNModel,self).__init__()
        self.l1 = nn.Linear(10, 32)
        self.l2 = nn.Linear(32, 64)
        self.l3 = nn.Linear(64, 64)
        self.l4 = nn.Linear(64, 32)
        self.l5 = nn.Linear(32, 93)
        
    def forward(self,x):
        # input data : ( n , 1 , 28 , 28 )
        x = x.view(-1, 10) # Flatten : ( n , 784 )
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [4]:
model = NNModel()


criterion = nn.L1Loss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.5)

model= model.to(device)

In [5]:
def train(epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)
        output = model(data)
        
        optimizer.zero_grad()

        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx%50==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.data))

In [6]:
def test():
    model.eval()
    test_loss=0
    correct=0
    for data,target in test_loader:
        data = data.to(device)
        target = target.to(device)

        output = model(data)
        test_loss += criterion(output,target).data
        pred = output
        # correct += pred.eq(target.data.view_as(pred)).sum()
        
        correct += abs(pred.sub(target.data.view_as(pred))).sum()
        
        test_loss /= len(test_loader.dataset)  
    print('Test set: Average loss: {:.9f}, Accuracy: {:.3f}%'.format(
        test_loss, 100. - correct/ len(train_loader) - 10))


    '''
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset))) 
    '''

In [7]:
for epoch in range(1, 10):
    train(epoch)
    
savePath = "./output/test_model.pth"
torch.save(model.state_dict(), savePath)
test()

Train Epoch: 1 [0/200 (0%)]	Loss: 0.196303
Train Epoch: 1 [50/200 (25%)]	Loss: 0.122051
Train Epoch: 1 [100/200 (50%)]	Loss: 0.294289
Train Epoch: 1 [150/200 (75%)]	Loss: 0.074254
Train Epoch: 2 [0/200 (0%)]	Loss: 0.072856
Train Epoch: 2 [50/200 (25%)]	Loss: 0.265890
Train Epoch: 2 [100/200 (50%)]	Loss: 0.057866
Train Epoch: 2 [150/200 (75%)]	Loss: 0.053067
Train Epoch: 3 [0/200 (0%)]	Loss: 0.099249
Train Epoch: 3 [50/200 (25%)]	Loss: 0.041735
Train Epoch: 3 [100/200 (50%)]	Loss: 0.050792
Train Epoch: 3 [150/200 (75%)]	Loss: 0.035625
Train Epoch: 4 [0/200 (0%)]	Loss: 0.043643
Train Epoch: 4 [50/200 (25%)]	Loss: 0.040988
Train Epoch: 4 [100/200 (50%)]	Loss: 0.113522
Train Epoch: 4 [150/200 (75%)]	Loss: 0.113107
Train Epoch: 5 [0/200 (0%)]	Loss: 0.166884
Train Epoch: 5 [50/200 (25%)]	Loss: 0.131918
Train Epoch: 5 [100/200 (50%)]	Loss: 0.206095
Train Epoch: 5 [150/200 (75%)]	Loss: 0.013377
Train Epoch: 6 [0/200 (0%)]	Loss: 0.024136
Train Epoch: 6 [50/200 (25%)]	Loss: 0.164716
Train Epoch: